### 误差反向传播法
- 使用**计算图**可以通过反向传播高效计算导数。
- 比较数值微分和误差反向传播法的结果，可以确认误差反向传播法的实现是否正确。
- 计算图的反向传播从右往左传播信号，反向传播的计算顺序是：先将节点的输入信号乘以节点的局部导数(偏导数)，然后再传递给下一个节点。
- **加法节点**的反向传播只是将输入信号输出到下一个节点。
  <div align="center"><img src="data_mk/mk-2025-11-03-23-08-48.png" width="75%" style="margin: 20px 0;"></div>
- **乘法节点**的反向传播会乘以输入信号的翻转值。<mark>实现乘法节点的反向传播时，要保存正向传播的输入信号。</mark>
  <div align="center"><img src="data_mk/mk-2025-11-03-23-15-39.png" width="75%" style="margin: 20px 0;"></div>

In [ ]:
# ==========================================
# 乘法层（Multiplication Layer）
# ==========================================
class MulLayer:
    def __init__(self):
        # 保存正向传播时输入的值（用于反向传播）
        # 在计算图中，一个节点的输出往往依赖于输入的值，
        # 因此在反向传播阶段需要将这些输入值重新取出使用。
        self.x = None
        self.y = None

    def forward(self, x, y):
        """
        前向传播 (Forward propagation)
        输入: x, y
        输出: out = x * y
        """
        # 将输入保存下来，方便反向传播使用
        self.x = x
        self.y = y
        # 乘法层的输出
        out = x * y
        return out

    def backward(self, dout):
        """
        反向传播 (Backward propagation)
        dout: 来自上游层的梯度 (∂L/∂out)

        计算目标：
            根据链式法则：
                dx = ∂L/∂x = ∂L/∂out * ∂out/∂x = dout * y
                dy = ∂L/∂y = ∂L/∂out * ∂out/∂y = dout * x
        """
        dx = dout * self.y  # 反向传播到 x 的梯度
        dy = dout * self.x  # 反向传播到 y 的梯度
        return dx, dy


# ==========================================
# 加法层（Addition Layer）
# ==========================================
class AddLayer:
    def __init__(self):
        # 加法层没有需要保存的参数，因为加法的导数恒为 1
        pass

    def forward(self, x, y):
        """
        前向传播:
        输入: x, y
        输出: out = x + y
        """
        out = x + y
        return out

    def backward(self, dout):
        """
        反向传播:
        dout: 来自上游层的梯度 (∂L/∂out)

        根据链式法则：
            out = x + y
            ∂out/∂x = 1, ∂out/∂y = 1
        所以：
            dx = dout * 1 = dout
            dy = dout * 1 = dout
        """
        dx = dout  # 对 x 的梯度
        dy = dout  # 对 y 的梯度
        return dx, dy

In [ ]:
import numpy as np

# ==========================================
# ReLU层（Rectified Linear Unit 激活层）
# ==========================================
class Relu:
    def __init__(self):
        # mask 是一个布尔数组，用来记录哪些神经元在前向传播中被“屏蔽”（即输入<=0）
        # 在反向传播时，被屏蔽的神经元梯度会被置为0，不参与更新。
        self.mask = None

    def forward(self, x):
        """
        前向传播 (Forward propagation)
        ReLU函数定义为：f(x) = max(0, x)
        作用：将输入中小于等于0的值截断为0，大于0的部分保持不变。

        ReLU的意义：
            - 引入非线性，使神经网络具备逼近复杂函数的能力；
            - 相比 Sigmoid/Tanh，ReLU 不会饱和，计算简单，收敛更快。
        """
        # mask 标记出小于等于0的位置
        self.mask = (x <= 0)

        # 将输入复制一份，避免原数据被修改
        out = x.copy()

        # 将 mask 为 True 的位置（即 x<=0）置为 0
        out[self.mask] = 0

        return out

    def backward(self, dout):
        """
        反向传播 (Backward propagation)
        dout: 来自上游层的梯度 (∂L/∂out)

        对于 ReLU:
            f(x) = max(0, x)
            当 x > 0 时，∂f/∂x = 1
            当 x <= 0 时，∂f/∂x = 0

        因此：
            dx = dout * (x > 0)
        """
        # 对 mask 中为 True 的（即 x<=0）梯度置 0
        dout[self.mask] = 0

        # 反向传播后的梯度（dx = dout * mask）
        dx = dout

        return dx


# ==========================================
# Sigmoid层（Sigmoid 激活层）
# ==========================================
class Sigmoid:
    def __init__(self):
        # 保存前向传播的输出值，在反向传播时使用
        self.out = None

    def forward(self, x):
        """
        前向传播:
        Sigmoid函数定义为：
            f(x) = 1 / (1 + exp(-x))

        特性：
            - 将输入“压缩”到 (0, 1) 区间；
            - 常用于输出概率或二分类任务；
            - 在深层网络中容易出现梯度消失问题（因为导数在饱和区接近0）。
        """
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        """
        反向传播:
        Sigmoid函数的导数为：
            f'(x) = f(x) * (1 - f(x))

        利用链式法则：
            dx = dout * f'(x)
               = dout * (1.0 - self.out) * self.out
        """
        dx = dout * (1.0 - self.out) * self.out
        return dx


### Sigmoid层计算图
<div align="center">   <img src="data_mk/mk-2025-11-04-20-08-59.png" width="75%" style="margin: 20px 0;" /> </div>

$$
\begin{align*}
\frac{\partial L}{\partial y}y^2\exp(-x)
&= \frac{\partial L}{\partial y}\frac{1}{(1+\exp(-x))^2}\exp(-x) \\
&= \frac{\partial L}{\partial y}\frac{1}{1+\exp(-x)}\frac{\exp(-x)}{1+\exp(-x)} \\
&=\frac{\partial L}{\partial y}y(1-y)
\end{align*}
$$


### 以矩阵为对象的反向传播
<div align="center">   <img src="data_mk/mk-2025-11-04-20-36-52.png" width="75%" style="margin: 20px 0;" /> </div>

$$
\bm{X} = (x_0,x_1,...,x_n)
$$
$$
\frac{\partial L}{\partial \bm{X}} = (\frac{\partial L}{\partial x_0},\frac{\partial L}{\partial x_1},...,\frac{\partial L}{\partial x_n})
$$

In [ ]:
import numpy as np

class Affine:
    """
    Affine层（全连接层 / 仿射层）
    作用：实现 y = xW + b 的前向传播，并能计算反向传播的梯度。
    该层通常位于神经网络的隐藏层或输出层中，用于线性变换输入特征。
    """
    def __init__(self, W, b):
        # 权重参数 W: (输入维度, 输出维度)
        # 偏置参数 b: (输出维度,)
        self.W = W
        self.b = b
        
        # 记录前向传播的输入（用于反向传播）
        self.x = None
        
        # 保存输入的原始形状（例如卷积层输出是4维的，需要还原）
        self.oringinal_x_shape = None
        
        # 存放反向传播时计算得到的梯度
        self.dW = None  # 对权重的梯度
        self.db = None  # 对偏置的梯度

    def forward(self, x):
        """
        前向传播：计算仿射变换输出
        y = xW + b
        """
        # -----------------【语法讲解】-----------------
        # x.reshape(x.shape[0], -1)
        # 含义：将输入x（可能是多维数据，如图像）拉平成二维矩阵。
        #       例如输入(batch_size, channels, height, width)
        #       -> 输出(batch_size, channels*height*width)
        # reshape 中 -1 表示“自动推导”剩余维度大小
        # 这样做是为了与权重矩阵 W (二维) 相乘。
        # ---------------------------------------------
        self.oringinal_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        
        # 仿射变换：矩阵乘法 + 偏置
        # 对应深度学习中的线性层 y = xW + b
        out = np.dot(self.x, self.W) + self.b
        return out

    def backward(self, dout):
        """
        反向传播：
        已知上游梯度 dout，计算该层输入 x、权重 W、偏置 b 的梯度
        """
        # dout: (batch_size, 输出维度)
        # W: (输入维度, 输出维度)

        # 输入梯度 dx = dout · W^T
        dx = np.dot(dout, self.W.T)

        # 权重梯度 dW = x^T · dout
        # 这对应损失函数对权重参数的偏导
        self.dW = np.dot(self.x.T, dout)

        # -----------------【语法讲解】-----------------
        # np.sum(dout, axis=0)
        # 含义：对每个输出维度方向求和，得到每个偏置的梯度。
        # 因为偏置 b 是对所有样本共享的，所以要把 batch 内的梯度加总。
        # 举例：若 dout.shape = (100, 10)
        # 则 db.shape = (10,)
        # ---------------------------------------------
        self.db = np.sum(dout, axis=0)

        # -----------------【语法讲解】-----------------
        # dx.reshape(*self.oringinal_x_shape)
        # 含义：将 dx 从二维矩阵还原为输入的原始形状。
        # 星号 * 是“解包”操作符，
        # 比如 original_x_shape=(100,1,28,28)，
        # 等价于 dx.reshape(100,1,28,28)
        # 这样便可在卷积层与全连接层之间无缝衔接。
        # ---------------------------------------------
        dx = dx.reshape(*self.oringinal_x_shape)

        # 返回输入的梯度
        return dx


### Softmax层
一、Softmax 层的作用

Softmax 层常用在**分类任务的输出层**，
它的作用是将网络输出的“原始得分”（logits）转化为**概率分布**，
使得每个类别的输出都在 ([0,1]) 之间，并且所有类别的概率之和为 1。

二、数学表达式

设网络的最后一层输出为一个向量：
$$
\mathbf{a} = [a_1, a_2, ..., a_K]
$$
其中$K$是类别数（比如 10 分类任务中 $K=10$）。

Softmax 的定义为：
$$
y_i = \frac{\exp(a_i)}{\sum_{j=1}^{K} \exp(a_j)}
$$

所以输出 $y_i$ 表示第 $i$ 类的预测概率。

三、为什么要用 Softmax？

网络输出的$a_i$ 一般是任意实数（可能为负数），
直接拿来代表概率显然不合理。

Softmax 有两个关键特性：

1. **非负性**：$\exp(a_i) > 0$，保证输出全为正；
2. **归一化**：分母求和后让所有概率加起来等于 1。

因此 Softmax 的输出就是一个概率分布，非常适合分类问题。


四、与交叉熵误差（Cross Entropy Error）的结合

如果只用 Softmax，还不能度量预测与真实标签的差距。
因此在训练中，我们把它和交叉熵损失函数结合：

$$
L = - \sum_{i} t_i \log(y_i)
$$
其中：

* $t_i$ 是真实标签（通常是 one-hot 向量）
* $y_i$ 是 Softmax 输出的预测概率
<div align="center">

| 类别 | One-hot 表示   |
| -- | ------------ |
| 0  | [1, 0, 0, 0] |
| 1  | [0, 1, 0, 0] |
| 2  | [0, 0, 1, 0] |
| 3  | [0, 0, 0, 1] |

</div>
这个公式的含义：

> 当真实类别 $t_i=1$ 时，损失只取该类对应的 $-\log(y_i)$。

所以：

* 如果预测概率接近 1，损失趋近于 0；
* 如果预测概率接近 0，损失趋近于无穷大；
  → 网络会被“惩罚”得更厉害。

五、Softmax-with-Loss 层的反向传播

这层在反向传播时有个非常重要的性质（公式推导后可以简化）：

$$
\frac{\partial L}{\partial a_i} = y_i - t_i
$$

也就是说，**Softmax + Cross Entropy 的组合可以直接得到梯度**，
从而避免显式计算 Softmax 的复杂导数，
这也是它们总是一起实现的原因。


In [ ]:
import numpy as np

# ============================================================
# Softmax 函数：将神经网络的输出（logits）转换为概率分布
# ============================================================
def softmax(x):
    # ------------------------ 多样本（二维输入）情况 ------------------------
    # x.shape = (batch_size, 类别数)
    if x.ndim == 2:
        # 为了方便按列计算 softmax，先转置
        x = x.T
        
        # 数值稳定化处理（防止指数运算时溢出）
        # 减去每一列的最大值，不影响相对概率，但能防止 exp() 溢出为无穷大
        x = x - np.max(x, axis=0)

        # softmax 计算公式：exp(x_i) / Σ exp(x_j)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)

        # 最后再转置回来，保持与输入形状一致
        return y.T 

    # ------------------------ 单样本（一维输入）情况 ------------------------
    # 同样进行溢出对策（数值稳定化）
    x = x - np.max(x)
    # softmax 函数输出：每个元素代表该类别的预测概率
    return np.exp(x) / np.sum(np.exp(x))


# ============================================================
# 交叉熵误差（Cross Entropy Error）
# 用于衡量预测概率分布 y 与真实分布 t 的差异
# ============================================================
def cross_entropy_error(y, t):
    # ------------------------ 统一为二维处理 ------------------------
    # 若输入为单一样本（一维），则转换为二维以便批量计算
    if y.ndim == 1:
        t = t.reshape(1, t.size)   # t: (类别数,) -> (1, 类别数)
        y = y.reshape(1, y.size)   # y: (类别数,) -> (1, 类别数)
        
    # ------------------------ One-hot 标签处理 ------------------------
    # 若 t 与 y 形状相同（说明 t 是 one-hot 向量）
    # 则取 argmax 得到每个样本真实类别的索引
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    # ------------------------ 损失计算 ------------------------
    batch_size = y.shape[0]
    # 取出每个样本中真实类别对应的预测概率 y[i, t[i]]
    # 加上一个极小值 1e-7 防止 log(0) 导致数值错误
    # 最后取负号并对整个 batch 求平均
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


# ============================================================
# Softmax-with-Loss 层
# 结合了 Softmax 激活函数 与 交叉熵损失函数
# 前向传播：计算概率与损失
# 反向传播：计算梯度（简化为 y - t）
# ============================================================
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 损失值（标量）
        self.y = None     # softmax 输出（概率分布）
        self.t = None     # 真实标签（监督数据）

    # ------------------------ 前向传播 ------------------------
    def forward(self, x, t):
        """
        x: 神经网络的输出（logits），形状 (batch_size, 类别数)
        t: 监督标签，可以是 one-hot 向量或类别索引
        """
        self.t = t
        # 计算 softmax 概率分布
        self.y = softmax(x)
        # 计算交叉熵损失
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    # ------------------------ 反向传播 ------------------------
    def backward(self, dout=1):
        """
        dout: 上游传来的梯度（通常为1，因为 loss 是标量）
        返回：dx —— 当前层输入的梯度，用于传给前一层
        """
        batch_size = self.t.shape[0]

        # ---------------- One-hot 标签情况 ----------------
        # 结合 Softmax 与 CrossEntropy 的反向传播有个重要结论：
        #   ∂L/∂x = (y - t) / batch_size
        # 这是由于两者结合后梯度极大简化，避免了显式计算 softmax 的复杂导数
        """
        self.t.size == self.y.size说明监督标签 t 是 one-hot 向量（和 y 的形状一样）。
        例如：
        y = [[0.1, 0.7, 0.2],   # 第1个样本的预测概率
             [0.8, 0.1, 0.1]]   # 第2个样本的预测概率
        t = [[0, 1, 0],         # 第1个样本真实类别为 1
             [1, 0, 0]]         # 第2个样本真实类别为 0
        """
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        # ---------------- 标签为类别索引情况 ----------------
        # 若 t 不是 one-hot（例如 t = [2, 0, 1]）
        # 则需要手动构造与之对应的梯度
        else:
            dx = self.y.copy()  # 复制预测概率
            # 对于真实类别 t[i]，其梯度为 (y_i - 1)
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        # 返回输入梯度（供前层使用）
        return dx

